# 3.1 Milvus 在图片搜索中的应用


Milvus 在 RAG（Retrieval-Augmented Generation，检索增强生成）系统中，扮演着“外部知识库”的核心角色。它的主要任务是将经过处理的文档内容转化为高维语义向量，并高效地进行存储与检索。

具体来说，RAG 系统会先将原始文档分成多个文本块，并利用嵌入模型将每个文本块转化为一个语义向量。Milvus 作为向量数据库，将这些文本块连同它们的向量及元数据一同保存。当用户提出问题时，系统会将问题同样转化为向量，并通过 Milvus 快速检索出最相关的文本块。

这些被检索到的文本块，作为知识依据和上下文，被输入到大语言模型中，用于进一步生成符合实际、内容可靠的答案。得益于 Milvus 的高性能向量检索能力，即便面对百万甚至亿级的文档数据，RAG 系统也能在极短时间内给出精准的知识响应。

简而言之，Milvus 是 RAG 体系中的“知识中枢”，保障了外部知识的有效扩展与按需调用。它让大模型生成不再依赖记忆和猜测，而是基于真实的、可追溯的语义资料。这也是现代智能问答系统可信、专业和可扩展的基础之一。



在本章中，我们将系统性回顾 RAG（Retrieval-Augmented Generation，检索增强生成）的核心技术流程，帮助大家从整体上理解这一技术架构是如何工作的。在后续的实验中，我们将基于这些环节逐步搭建一个具备真实问答能力的智能系统。

RAG 的提出，源于传统大语言模型（LLM）存在的两大问题：

1. **记忆范围有限**：模型上下文窗口有限，无法直接处理大体量知识。
2. **容易“胡说”**：模型会根据语义猜测输出内容，可能编造事实（Hallucination）。

而 RAG 的核心思想是：\*\*将检索模块引入生成过程，让模型在生成答案前先“查资料”。\*\*这样可以大幅提升回答的相关性和准确性，尤其适用于企业文档问答、知识库助手、法律法规摘要、技术支持等场景。

RAG 系统通常包含以下六个关键步骤：

```
文档加载 → 分块（Chunking）→ 向量化（Embedding）→ 存储 → 检索 → 生成
```

下面我们逐一简要介绍这六个步骤的意义与要点。

---

### 1️⃣ 文档加载（Document Loading）

这是整个流程的起点。我们需要先将企业内部的知识内容加载进系统中。常见的文档格式包括 `.pdf`、`.txt`、`.md`、`.docx` 等，信息来源可以是本地文件、网页、数据库，甚至是云端文档（如 Notion、Confluence、S3 等）。

在实践中，推荐使用 LangChain 中的 `DocumentLoader` 工具，结合 `Unstructured`、`PyMuPDF`、`pdfplumber` 等库，帮助我们将文档加载为标准的文本数据结构（Document List）。

---

### 2️⃣ 分块（Chunking）

原始文档往往篇幅较长，不适合直接向量化，因此需要进行“分块”处理，将长文档拆解为数百字的短文本块（chunk）。

常见策略包括：

* 固定大小拆分（如每 500 字一个 chunk）
* 滑动窗口拆分（如每 500 字，重叠 100 字）
* 按段落或标题拆分（保留语义完整性）

Chunking 的策略将直接影响后续检索效果：块太小语义不完整，块太大可能模糊焦点。因此，chunk size 和 overlap 是需要调优的重要参数。

---

### 3️⃣ 向量化（Embedding）

每个文本块需要通过语言模型转化为向量，这个向量就是模型所理解的“语义表示”。

常用的嵌入模型包括：

* OpenAI 的 `text-embedding-3-small`
* Sentence Transformers 的 `all-MiniLM-L6-v2`
* Amazon Bedrock 提供的 `Titan Embeddings`
* Cohere 的 `embed-english-light-v3`

生成的向量通常是 384、768 或 1536 维的浮点数组，表示该文本块的语义位置。后续我们可以用这些向量做相似度比较（如余弦相似度）来实现语义检索。

---

### 4️⃣ 存储（Storage）

生成的向量需要持久化保存，以供随时检索。我们通常使用**向量数据库**来完成这项任务。

主流的向量数据库包括：

* **Milvus**（支持亿级数据，性能强）
* FAISS（轻量高效，适合本地测试）
* Pinecone（托管型，简单易用）
* Weaviate（支持 GraphQL 和语义推理）

这些数据库不仅能保存向量本身，还能附带原始文本和元数据（如文档名、段落位置等），方便在回答中进行引用。

---

### 5️⃣ 检索（Retrieval）

当用户提出问题时，系统会先将问题进行向量化，然后与数据库中的向量进行相似度搜索，找出最相关的文本块。这个过程就是“语义检索”。

为了提升检索准确性，很多系统还会使用混合检索（Hybrid Search）技术，结合关键词匹配（如 BM25）与语义匹配，或加入 metadata filter 进行限制筛选。

检索结果通常返回 Top-K（如最相关的 3\~5 段）文本块，作为生成模块的输入上下文。

---

### 6️⃣ 生成（Generation）

最后，将检索到的文本块与用户提问一并输入到大语言模型中，由模型基于上下文生成最终答案。

你可以自定义 prompt 模板，如：

```
请根据以下内容回答问题：
{context}

问题：{question}
```

LLM 会参考检索内容进行生成，从而提升准确性、降低幻觉率。

---

通过这六个步骤，我们构建出了一个完整的“从知识库中查找信息并生成回答”的智能问答系统。接下来的实验中，我们将以这一流程为基础，实战搭建一个基于 LangChain 和 Milvus 的 RAG 系统。

好的，以下是完整约 1000 字的章节内容，围绕「Milvus 在 RAG 中的角色」，适合作为实验手册中的一章开头，语言清晰、结构合理、技术导向明确：

---

## Milvus 在 RAG 中的角色

在构建一个基于 RAG（Retrieval-Augmented Generation，检索增强生成）的问答系统时，很多人第一时间关注的是大语言模型（LLM）的回答能力，然而，真正决定答案质量的往往是模型“看到了什么”。这些“上下文信息”的提供，正是向量数据库的任务。而在众多向量数据库中，**Milvus** 是最常被使用、也最适合工程化落地的一种解决方案。

本章将带大家深入了解：Milvus 在整个 RAG 架构中究竟承担了怎样的角色？它是如何提升问答系统的准确性、效率与可扩展性的？我们又该如何正确地使用它？

---

### 🧩 Milvus 是什么？

Milvus 是一个开源、高性能的向量数据库系统，专门用于处理大规模的向量数据。与传统的关系型数据库不同，Milvus 不擅长做精确查找，而是专注于语义检索——也就是**给出一个查询向量，找出与之“语义相似”的数据项**。

在 NLP 场景中，我们将文本通过嵌入模型转化为向量表示，这些向量就代表了文本的语义。Milvus 能够快速地在成千上万个文本向量中，找出与某个问题向量最接近的几条内容，从而实现真正的“先查资料，再作答”。

---

### 🧠 在 RAG 流程中的职责

Milvus 在 RAG 系统中主要承担两项关键任务：

#### 1. 存储文本块及其语义向量

RAG 系统的第一步是加载文档，并将其分成若干小段（chunk）。每一段文本都会被一个嵌入模型（如 `all-MiniLM-L6-v2` 或 OpenAI Embedding 模型）转化为一个高维向量。

这些向量不能直接存在内存中，否则随着文档数量增长，系统将无法负载。此时，Milvus 作为向量数据库登场，**它不仅存储这些向量，还一并保存原始文本和元数据（如文件名、页码、段落编号等）**，形成一个完整的语义检索库。

#### 2. 快速检索与用户问题相似的文本块

当用户提出一个自然语言问题时，该问题也会被同样的嵌入模型转化为一个语义向量。这个查询向量会被发送给 Milvus，系统会在存储库中查找与之最接近的 Top-K 向量所对应的文本块。

这一步检索结果将作为**上下文**被喂给 LLM（如 GPT、Claude），成为其回答问题的依据。换句话说，Milvus 决定了 LLM“看到了什么”，而 LLM 决定了“怎么说”。

---

### ⚡ 为什么选择 Milvus？

虽然市面上有许多向量数据库，比如 FAISS、Pinecone、Weaviate 等，但 Milvus 拥有一系列特点，使其特别适合构建工业级的 RAG 系统：

* **高性能**：基于 FAISS/HNSW/IVF 等高效检索算法，支持亿级向量的快速查询。
* **结构化过滤能力**：支持向量检索 + metadata（结构化标签）联合查询，比如“只检索标签为技术文档的内容”。
* **部署灵活**：支持本地部署（Docker/K8s）与云托管（Zilliz Cloud）。
* **与 LangChain 深度集成**：可作为 `VectorStore`，直接参与 RAG 工作流。
* **活跃社区支持**：拥有完善文档、示例项目和 API，学习成本低。

这些特性使得 Milvus 不仅适合初学者实验项目，也可以轻松扩展到企业级应用场景。

---

### 🧪 总结：Milvus 是 RAG 的“记忆中枢”

在 RAG 的六步流程中：

```
文档加载 → 分块（Chunking）→ 向量化（Embedding）→ 存储（Milvus）→ 检索 → 生成
```

Milvus 虽不是回答问题的“主角”，却是整个系统的“记忆中枢”。它让大语言模型具备了“短期记忆之外的长期记忆”，从而能够处理结构化文档、非结构化知识、复杂报告等超出上下文窗口的海量信息。

RAG 的本质是连接检索与生成，而 Milvus 就是这一连接中的桥梁，它保证了系统的可扩展性、准确性与实时性。

---

在接下来的章节中，我们将正式上手：使用 LangChain 将向量数据写入 Milvus，并完成一次真实的问题检索与上下文生成流程。a


# RAG with LangChain & Milvus: Sentence Transformers 本地模型 Demo

> 适合无 OpenAI Key、企业内网、敏感知识库等场景。

**依赖安装**
```bash
pip install langchain pymilvus sentence-transformers ollama
```
Milvus 推荐 docker 一键启动，
文档示例路径：docs/**.md


In [4]:
# 0. 导入依赖 & 环境准备
import os
from langchain.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, list_collections

In [5]:
# 选择本地向量模型
embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# # ===== 1. 加载文档并分块 =====
# loader = TextLoader("docs/enterprise_guide.txt", encoding="utf-8")
# documents = loader.load()
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=80)
# docs = text_splitter.split_documents(documents)
# print(f'文档共分块: {len(docs)} 个')


# 1. 加载 doc 目录下所有 .md 文件，使用 TextLoader 解析文本内容
loader = DirectoryLoader("docs", glob="**/*.md", loader_cls=TextLoader)
documents = loader.load()
print(f'共加载文档数：{len(documents)}')

# 2. 使用 RecursiveCharacterTextSplitter 分块
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=80)
docs = text_splitter.split_documents(documents)
print(f'文档共分块: {len(docs)} 个')

共加载文档数：7
文档共分块: 52 个


In [6]:
# ===== 2. 文本向量化 =====
texts = [doc.page_content for doc in docs]
vectors = embed_model.embed_documents(texts)
print(f'已完成文本向量化，向量维度: {len(vectors[0])}')

已完成文本向量化，向量维度: 384


In [7]:
# 3. 连接 Milvus 并初始化 Collection
connections.connect(host="localhost", port="19530")
collection_name = "rag_demo_local"
dim = len(vectors[0])

if collection_name not in list_collections():
    fields = [
        FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
        FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=dim),
        FieldSchema(name="content", dtype=DataType.VARCHAR, max_length=2048),
    ]
    schema = CollectionSchema(fields)
    collection = Collection(name=collection_name, schema=schema)
    collection.create_index(
        field_name="embedding",
        index_params={"index_type": "IVF_FLAT", "metric_type": "L2", "params": {"nlist": 128}}
    )
else:
    collection = Collection(collection_name)

In [8]:
# ===== 4. 插入数据到 Milvus =====
milvus_data = [
    vectors,   # embedding
    texts,     # content
]
mr = collection.insert(milvus_data)
collection.load()
print(f'已插入 {len(texts)} 条分块')


已插入 52 条分块


In [11]:
# 5. 用户输入问题并检索相关内容
user_query = "懒猫微服DNS和代理冲突怎么办？"
query_vec = embed_model.embed_query(user_query)
search_params = {"metric_type": "L2", "params": {"nprobe": 10}}
results = collection.search(
    data=[query_vec],
    anns_field="embedding",
    param=search_params,
    limit=4,
    output_fields=["content"]
)
retrieved_chunks = [hit.entity.get("content") for hit in results[0]]
for idx, chunk in enumerate(retrieved_chunks, 1):
    print(f'相关片段{idx}：{chunk[:120]}...')

相关片段1：# 懒猫微服：完全DNS访问，和本地代理冲突了怎么办？

刚拿到懒猫微服的时候，了解到这个机器完全使用DNS 来访问是很吃惊的。拒不完全使用经验，大概是机器里部署了一套私有的 DNS server，然后广播到整个局域网。而公网上的则是hei...
相关片段2：懒猫微服和传统的NAS又很大的不同，如果作为小白玩家可以很快上手，当做 Sass 服务来用。但对于专业玩家，总有一种技术的强迫症，总用抽丝剥茧，从 Saas 一点点解析到 Iass，然后一点把懒猫编程能够公开访问的私有云。

比如网络。可以...
相关片段3：### 3）DOMAIN-KEYWORD

- 匹配**包含某个关键词的所有域名**。
- 举例：`apple` → `apple.com`、`gs.apple.com`、`appleabc.xyz` 都会命中。

### 4）IP-CID...
相关片段4：而最终落到配置文件上就是这样的。（之前写DOMAIN-SUFFIX,*.lazycat.cloud,DIRECT）一直不生效。

```
rules:
- DOMAIN-SUFFIX,lazycat.cloud,DIRECT
- DOMAI...


In [12]:
context = '\n'.join(retrieved_chunks)
prompt = f'已知资料如下：\n{context}\n\n请根据以上资料回答用户问题：{user_query}'

In [14]:
from ollama import Client

context = '\n'.join(retrieved_chunks)
# user_query = "我和她在哪里认识的"

prompt = f"已知资料如下：\n{context}\n\n请根据以上资料回答用户问题：{user_query}"

client = Client()

response = client.chat(
    model='deepseek-r1:7b',  # 例如 'llama2' 或其他
    messages=[{'role': 'user', 'content': prompt}]
)

print(response['message']['content'])


<think>
嗯，我现在遇到了一个问题，就是懒猫微服使用的是完全DNS访问，但这个机器部署了一套私有DNS服务器，覆盖了局域网。同时，公网上的解析则由heiyu.space提供。这样在访问的时候，可能需要先通过私有DNS进行解析，如果不在局域网内，就需要走互联网上的DNSPod。

我记得用户提到，在上传文件到懒猫网盘时速度很慢，怀疑是用了代理服务器导致的延迟。那怎么解决呢？根据资料，可以考虑设置白名单，阻止代理访问。但问题在于，很多代理服务器都是基于域名来过滤流量的，所以需要修改这些DNS规则。

资料里提到有几种DNS规则类型：DOMAIN、DOMAIN-SUFFIX、DOMAIN-KEYWORD和IP-CIDR。 DOMAIN-SUFFIX匹配所有以指定后缀结尾的域名，比如`.lazycat.cloud`会匹配所有类似结构的域名。而DOMAIN-KEYWORD则匹配包含特定关键词的域名。

用户的问题是懒猫微服DNS与代理冲突，导致无法正常访问。可能是因为当访问懒猫微服的时候，先通过私有DNS解析到局域网IP，然后被代理服务器拦截了流量，导致延迟甚至丢包。

解决方法应该是修改DNS规则，让代理只允许特定的域名或者IP段通过。例如：

1. **设置DNS规则**：在局域网中添加DNS记录，允许仅针对懒猫微服的域名或IP段，这样当用户访问时不会被代理拦截。
2. **静态IP配置**：如果可能的话，直接将机器分配到内网上的静态IP地址，这样可以直接走上级路由，避免代理的介入。

资料中提到，如果使用nmtui这样的DNS配置工具，可以手动设置静态IP。这应该是一个可行的办法，因为静态IP bypassesDNS解析，直接连接到内部服务。

步骤可能是：
- 配置静态IP：将机器分配到一个内网静态IP地址。
- 修改DNS规则：添加DNS记录，允许仅懒猫微服相关的域名通过，或者关闭DNS转发功能，只允许本地域名使用局域网的解析。

这样既保留了懒猫微服作为SaaS服务的优势，又避免了代理服务器带来的延迟问题。需要注意的是，在修改DNS规则时，要确保不影响其他正常的DNS查询。
</think>

针对懒猫微服DNS和代理冲突的问题，可以采取以下步骤进行配置：

1. **使用静态IP**：
   - 将机器分配到内网上的静态IP地址。这样可以直接连接到内部服务

你提到的“做RAG的东西拆chunk，然后登向量，做召回，给大模型，整理回答response给end user”的流程，可以这样解释：

## RAG（Retrieval-Augmented Generation）流程简述

1. **拆Chunk（分块）**  
   首先，将大块文本或文档拆分成多个小块（chunk），每个chunk包含相对独立且语义完整的信息单元。分块策略可以是固定大小分块、基于文档结构分块、递归分块或基于语义的分块。目的是让每个chunk既不太大以致难以处理，也不太小以致丢失上下文[6][7]。

2. **登向量（向量化）**  
   将这些文本chunk通过预训练的嵌入模型（如BERT、Sentence Transformers等）转换成向量表示，存储在向量数据库中。向量化使得文本的语义信息可以用数字形式表达，方便后续的相似度计算和检索[5][6]。

3. **做召回（检索相关chunk）**  
   当用户发出查询时，系统也将查询转成向量，并在向量库中进行相似度搜索，召回与查询最相关的若干个chunk。召回的chunk数量（K值）需要合理设置，过少可能导致信息不全，过多则可能超出大模型输入限制或引入噪声[1][3][5]。

4. **给大模型（LLM）**  
   将召回的相关chunk与用户的查询一起，作为上下文输入给大语言模型。大模型基于这些外部知识和自身训练的语言能力，生成更加准确、丰富且符合用户需求的回答[2][5]。

5. **整理回答（生成响应）**  
   大模型结合召回内容和提示词，生成最终的回答文本，返回给用户。为了提升回答质量，通常会对召回的文本进行智能拼装，避免超长截断，并通过提示词设计和模型参数调整优化回答[1][4]。

---

## 具体说明

- **为什么拆chunk？**  
  大模型一次性处理文本有长度限制，且长文本难以直接向量化和检索。拆分成合理大小的chunk，既保证语义完整，又方便快速检索和拼装[6][7]。

- **为什么登向量？**  
  向量化让文本和查询都变成可计算的数字形式，通过计算向量之间的相似度实现语义层面的匹配，而非简单关键词匹配，召回更精准[5]。

- **为什么做召回？**  
  召回阶段是从海量知识中筛选出最相关的信息，减少大模型的输入负担，保证回答基于最新、最相关的知识，而非仅依赖模型训练时的固有知识[1][3][5]。

- **为什么给大模型？**  
  大模型负责理解查询和召回内容的关系，综合生成自然、连贯、准确的回答，发挥其强大的语言理解和生成能力[2][4]。

- **为什么整理回答？**  
  召回的多个chunk可能存在冗余或冲突，整理和智能拼装能保证输入给大模型的上下文合理且不超长，从而提升回答质量和用户体验[1]。

---

总结来说，这个流程是RAG系统的核心：先把知识拆成小块并向量化，用户查询时快速召回相关块，再由大模型结合召回内容生成回答，既保证了知识的时效性和准确性，也发挥了大模型的语言生成优势[1][2][3][5][6][7]。

[1] https://help.aliyun.com/zh/model-studio/rag-optimization
[2] https://www.acorn.io/resources/learning-center/retrieval-augmented-generation/
[3] https://www.alibabacloud.com/help/en/model-studio/rag-optimization
[4] https://juejin.cn/post/7286912324593598522
[5] https://aws.amazon.com/what-is/retrieval-augmented-generation/
[6] https://zilliz.com/learn/guide-to-chunking-strategies-for-rag
[7] https://blog.csdn.net/m0_56299345/article/details/142325902
[8] https://blog.csdn.net/weixin_45312236/article/details/135763694
[9] https://cloud.tencent.com/developer/article/2501015
[10] https://www.cnblogs.com/ting1/p/18598176

